# Dog venues in Boston, MA

In [288]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Libraries imported.


In [2]:
!conda install -c conda-forge geopy --yes
#from geopy.geocoders import Nominatim # convert an address into latitude and longitude values


Solving environment: ...working... done

# All requested packages already installed.



In [3]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

In [101]:
CLIENT_ID = 'NAF0ED02AESTUBSNE5LS34O4SBSAGDR0NX2V04Y5VJOQLTPM' # your Foursquare ID
CLIENT_SECRET = 'CQ5H2IZSYIMNI4B1TCYYHOCUFEJMRMBFHPHG2ICKNUH5V5FT' # your Foursquare Secret
ACCESS_TOKEN = 'NYUYCST5B1MIDLB1Y5DS44K4ZOJMKQMX0HV2I5CP4ZSONWPX' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
print('access' + ACCESS_TOKEN)

Your credentails:
CLIENT_ID: NAF0ED02AESTUBSNE5LS34O4SBSAGDR0NX2V04Y5VJOQLTPM
CLIENT_SECRET:CQ5H2IZSYIMNI4B1TCYYHOCUFEJMRMBFHPHG2ICKNUH5V5FT
accessNYUYCST5B1MIDLB1Y5DS44K4ZOJMKQMX0HV2I5CP4ZSONWPX


#### Getting coordinates for Boston, MA

In [174]:
address = 'Boston, MA'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

42.3602534 -71.0582912


#### Searching for entries containing 'Park' near Boston, MA

In [335]:
# SEARCHING FOR PARKS NEAR BOSTON

search_query = ['Park']
limit = 500
radius = 7000
url = 'https://api.foursquare.com/v2/venues/search?categoryId=4bf58dd8d48988d163941735&client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit=500'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
url

"https://api.foursquare.com/v2/venues/search?categoryId=4bf58dd8d48988d163941735&client_id=NAF0ED02AESTUBSNE5LS34O4SBSAGDR0NX2V04Y5VJOQLTPM&client_secret=CQ5H2IZSYIMNI4B1TCYYHOCUFEJMRMBFHPHG2ICKNUH5V5FT&ll=42.3602534,-71.0582912&oauth_token=NYUYCST5B1MIDLB1Y5DS44K4ZOJMKQMX0HV2I5CP4ZSONWPX&v=20180604&query=['Park']&radius=7000&limit=500"

In [336]:
# GETTING THE JSON FILE

results_park = requests.get(url).json()
results_park

{'meta': {'code': 200, 'requestId': '60207b072e5249192ee9a1ea'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'venues': [{'id': '4ba50617f964a520f1d138e3',
    'name': 'North End Park',
    'location': {'address': 'Cross St.',
     'crossStreet': 'btwn North & Sudbury',
     'lat': 42.36248823184806,
     'lng': -71.05647696102726,
     'labeledLatLngs': [{'label': 'display',
       'lat': 42.36248823184806,
       'lng': -71.05647696102726}],
     'distance': 290,
     'postalCode': '02113',
     'cc': 'US',
     'city': 'Boston',
     'state': 'MA',
     'country': 'United States',
     'formattedAddress': ['Cross St. (btwn North & Sudbury)',
      'Boston, MA 02113']},
    'categories': [{'id': '4bf58dd8d48988d163941735',
      'name': 'Park',
      'pluralName': 'Parks',
      'shortName': 'Park',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/park_',
       'suffix': '.png'},
      'primary': True}],
 

#### Creating dataframe

In [337]:
# assign relevant part of JSON to venues
venues_park = results_park['response']['venues']

# tranform venues into a dataframe
dataframe_park = json_normalize(venues_park)
dataframe_park.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",False,4ba50617f964a520f1d138e3,Cross St.,US,Boston,United States,btwn North & Sudbury,290,"[Cross St. (btwn North & Sudbury), Boston, MA ...","[{'label': 'display', 'lat': 42.36248823184806...",42.362488,-71.056477,NaN,02113,MA,North End Park,v-1612741383,NaN
1,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",False,4ae3580af964a520c59321e3,Atlantic Ave.,US,Boston,United States,at Atlantic Ave.,569,"[Atlantic Ave. (at Atlantic Ave.), Boston, MA ...","[{'label': 'display', 'lat': 42.36102715769776...",42.361027,-71.051451,NaN,02110,MA,Christopher Columbus Park,v-1612741383,NaN
2,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",False,54a1d1ce498e248911805297,NaN,US,Boston,United States,NaN,448,"[Boston, MA]","[{'label': 'display', 'lat': 42.36072318013166...",42.360723,-71.052873,NaN,NaN,MA,Harbor Park,v-1612741383,NaN
3,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",False,4bcf72669854d13a8f13f64d,NaN,US,Charlestown,United States,Chelsea St,1143,"[Charlestown, MA 02129]","[{'label': 'display', 'lat': 42.37019699121477...",42.370197,-71.061781,NaN,02129,MA,Paul Revere Park,v-1612741383,NaN
4,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",False,4bd9bd3fd2cbc92844a3d1ad,Museum Way,US,Cambridge,United States,NaN,1332,"[Museum Way, Cambridge, MA 02141]","[{'label': 'display', 'lat': 42.36911761365681...",42.369118,-71.069175,NaN,02141,MA,North Point Park,v-1612741383,NaN


#### Cleaning up dataframe

In [338]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns_park = ['name', 'categories'] + [col for col in dataframe_park.columns if col.startswith('location.')] + ['id']
dataframe_filtered_park = dataframe_park.loc[:, filtered_columns_park]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered_park['categories'] = dataframe_filtered_park.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered_park.columns = [column.split('.')[-1] for column in dataframe_filtered_park.columns]

dataframe_filtered_park.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,North End Park,Park,Cross St.,US,Boston,United States,btwn North & Sudbury,290,"[Cross St. (btwn North & Sudbury), Boston, MA ...","[{'label': 'display', 'lat': 42.36248823184806...",42.362488,-71.056477,NaN,02113,MA,4ba50617f964a520f1d138e3
1,Christopher Columbus Park,Park,Atlantic Ave.,US,Boston,United States,at Atlantic Ave.,569,"[Atlantic Ave. (at Atlantic Ave.), Boston, MA ...","[{'label': 'display', 'lat': 42.36102715769776...",42.361027,-71.051451,NaN,02110,MA,4ae3580af964a520c59321e3
2,Harbor Park,Park,NaN,US,Boston,United States,NaN,448,"[Boston, MA]","[{'label': 'display', 'lat': 42.36072318013166...",42.360723,-71.052873,NaN,NaN,MA,54a1d1ce498e248911805297
3,Paul Revere Park,Park,NaN,US,Charlestown,United States,Chelsea St,1143,"[Charlestown, MA 02129]","[{'label': 'display', 'lat': 42.37019699121477...",42.370197,-71.061781,NaN,02129,MA,4bcf72669854d13a8f13f64d
4,North Point Park,Park,Museum Way,US,Cambridge,United States,NaN,1332,"[Museum Way, Cambridge, MA 02141]","[{'label': 'display', 'lat': 42.36911761365681...",42.369118,-71.069175,NaN,02141,MA,4bd9bd3fd2cbc92844a3d1ad


#### Map of parks

In [339]:
venues_map_park = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around Boston


# add the parks as blue circle markers
for lat, lng, label in zip(dataframe_filtered_park.lat, dataframe_filtered_park.lng, dataframe_filtered_park.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map_park)

# display map
venues_map_park

#### Searching for entries containing 'pet'

In [340]:
# SEARCHING FOR PETS NEAR BOSTON

search_query = ['Pet']
limit = 500
radius = 7000
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit=500'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
url

"https://api.foursquare.com/v2/venues/search?client_id=NAF0ED02AESTUBSNE5LS34O4SBSAGDR0NX2V04Y5VJOQLTPM&client_secret=CQ5H2IZSYIMNI4B1TCYYHOCUFEJMRMBFHPHG2ICKNUH5V5FT&ll=42.3602534,-71.0582912&oauth_token=NYUYCST5B1MIDLB1Y5DS44K4ZOJMKQMX0HV2I5CP4ZSONWPX&v=20180604&query=['Pet']&radius=7000&limit=500"

In [341]:
# GETTING THE JSON FILE

results_pet = requests.get(url).json()
results_pet

{'meta': {'code': 200, 'requestId': '60207b1b13a18748e176d2ee'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'venues': [{'id': '5f84ad1ab70817260c693afc',
    'name': 'The Pet Vet - North End',
    'location': {'lat': 42.364488,
     'lng': -71.060868,
     'labeledLatLngs': [{'label': 'display',
       'lat': 42.364488,
       'lng': -71.060868}],
     'distance': 516,
     'postalCode': '02114',
     'cc': 'US',
     'city': 'Boston',
     'state': 'MA',
     'country': 'United States',
     'formattedAddress': ['Boston, MA 02114']},
    'categories': [{'id': '4d954af4a243a5684765b473',
      'name': 'Veterinarian',
      'pluralName': 'Veterinarians',
      'shortName': 'Veterinarians',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/medical_veterinarian_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1612741403',
    'hasPerk': False},
   {'id': '5697e553498e21a5b27ed54f',
    'name

#### Create a dataframe

In [342]:
# assign relevant part of JSON to venues
venues_pet = results_pet['response']['venues']

# tranform venues into a dataframe
dataframe_pet = json_normalize(venues_pet)
dataframe_pet.head()

,categories,delivery.id,delivery.provider.icon.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.name,delivery.url,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4d954af4a243a5684765b473', 'name': 'V...",NaN,NaN,NaN,NaN,NaN,NaN,False,5f84ad1ab70817260c693afc,NaN,US,Boston,United States,NaN,516,"[Boston, MA 02114]","[{'label': 'display', 'lat': 42.364488, 'lng':...",42.364488,-71.060868,02114,MA,The Pet Vet - North End,v-1612741403,NaN
1,"[{'id': '4d954af4a243a5684765b473', 'name': 'V...",NaN,NaN,NaN,NaN,NaN,NaN,False,5697e553498e21a5b27ed54f,5 Mystic View Rd,US,Everett,United States,NaN,4360,"[5 Mystic View Rd, Everett, MA 02149]","[{'label': 'display', 'lat': 42.398629, 'lng':...",42.398629,-71.068905,02149,MA,Banfield Pet Hospital,v-1612741403,NaN
2,"[{'id': '4bf58dd8d48988d100951735', 'name': 'P...",NaN,NaN,NaN,NaN,NaN,NaN,False,56045e14498e6d37759308a3,630 Fellsway,US,Medford,United States,NaN,5577,"[630 Fellsway, Medford, MA 02155]","[{'label': 'display', 'lat': 42.40683108115126...",42.406831,-71.083285,02155,MA,Pet Supplies Plus Medford,v-1612741403,NaN
3,"[{'id': '4bf58dd8d48988d100951735', 'name': 'P...",NaN,NaN,NaN,NaN,NaN,NaN,False,4bbf859fabf49521f1c8bd93,274 Southampton St,US,Boston,United States,NaN,3270,"[274 Southampton St, Boston, MA 02118]","[{'label': 'display', 'lat': 42.33245215, 'lng...",42.332452,-71.071146,02118,MA,Skipton Pet Center,v-1612741403,NaN
4,[],NaN,NaN,NaN,NaN,NaN,NaN,False,54cf294b498e696f2488e595,NaN,US,Boston,United States,NaN,52,"[Boston, MA]","[{'label': 'display', 'lat': 42.3600825, 'lng'...",42.360082,-71.058880,NaN,MA,Brandy Pete's,v-1612741403,NaN


In [343]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns_pet = ['name', 'categories'] + [col for col in dataframe_pet.columns if col.startswith('location.')] + ['id']
dataframe_filtered_pet = dataframe_pet.loc[:, filtered_columns_pet]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered_pet['categories'] = dataframe_filtered_pet.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered_pet.columns = [column.split('.')[-1] for column in dataframe_filtered_pet.columns]

dataframe_filtered_pet.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,The Pet Vet - North End,Veterinarian,NaN,US,Boston,United States,NaN,516,"[Boston, MA 02114]","[{'label': 'display', 'lat': 42.364488, 'lng':...",42.364488,-71.060868,02114,MA,5f84ad1ab70817260c693afc
1,Banfield Pet Hospital,Veterinarian,5 Mystic View Rd,US,Everett,United States,NaN,4360,"[5 Mystic View Rd, Everett, MA 02149]","[{'label': 'display', 'lat': 42.398629, 'lng':...",42.398629,-71.068905,02149,MA,5697e553498e21a5b27ed54f
2,Pet Supplies Plus Medford,Pet Store,630 Fellsway,US,Medford,United States,NaN,5577,"[630 Fellsway, Medford, MA 02155]","[{'label': 'display', 'lat': 42.40683108115126...",42.406831,-71.083285,02155,MA,56045e14498e6d37759308a3
3,Skipton Pet Center,Pet Store,274 Southampton St,US,Boston,United States,NaN,3270,"[274 Southampton St, Boston, MA 02118]","[{'label': 'display', 'lat': 42.33245215, 'lng...",42.332452,-71.071146,02118,MA,4bbf859fabf49521f1c8bd93
4,Brandy Pete's,None,NaN,US,Boston,United States,NaN,52,"[Boston, MA]","[{'label': 'display', 'lat': 42.3600825, 'lng'...",42.360082,-71.058880,NaN,MA,54cf294b498e696f2488e595


#### Create map for venues

In [344]:
venues_map_pet = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around Boston


# add the parks as blue circle markers
for lat, lng, label in zip(dataframe_filtered_pet.lat, dataframe_filtered_pet.lng, dataframe_filtered_pet.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map_pet)

# display map
venues_map_pet

#### Search for entries containing 'dog'

In [345]:
# SEARCHING FOR DOG NEAR BOSTON

search_query = ['Dog']
limit = 500
radius = 7000
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit=500'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
url

"https://api.foursquare.com/v2/venues/search?client_id=NAF0ED02AESTUBSNE5LS34O4SBSAGDR0NX2V04Y5VJOQLTPM&client_secret=CQ5H2IZSYIMNI4B1TCYYHOCUFEJMRMBFHPHG2ICKNUH5V5FT&ll=42.3602534,-71.0582912&oauth_token=NYUYCST5B1MIDLB1Y5DS44K4ZOJMKQMX0HV2I5CP4ZSONWPX&v=20180604&query=['Dog']&radius=7000&limit=500"

In [346]:
# GETTING THE JSON FILE

results_dog = requests.get(url).json()
results_dog

{'meta': {'code': 200, 'requestId': '60207b32865499558941cfb7'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'venues': [{'id': '40b28c80f964a520b5f71ee3',
    'name': 'Salty Dog Seafood Grille & Bar',
    'location': {'address': '206 Faneuil Hall Market Pl',
     'lat': 42.35993714563105,
     'lng': -71.05567659306708,
     'labeledLatLngs': [{'label': 'display',
       'lat': 42.35993714563105,
       'lng': -71.05567659306708}],
     'distance': 217,
     'postalCode': '02109',
     'cc': 'US',
     'city': 'Boston',
     'state': 'MA',
     'country': 'United States',
     'formattedAddress': ['206 Faneuil Hall Market Pl', 'Boston, MA 02109']},
    'categories': [{'id': '4bf58dd8d48988d1ce941735',
      'name': 'Seafood Restaurant',
      'pluralName': 'Seafood Restaurants',
      'shortName': 'Seafood',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/seafood_',
       'suffix': '.png'},
      'primary': True}],


#### Create a dataframe

In [347]:
# assign relevant part of JSON to venues
venues_dog = results_dog['response']['venues']

# tranform venues into a dataframe
dataframe_dog = json_normalize(venues_dog)
dataframe_dog.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d1ce941735', 'name': 'S...",False,40b28c80f964a520b5f71ee3,206 Faneuil Hall Market Pl,US,Boston,United States,NaN,217,"[206 Faneuil Hall Market Pl, Boston, MA 02109]","[{'label': 'display', 'lat': 42.35993714563105...",42.359937,-71.055677,02109,MA,Salty Dog Seafood Grille & Bar,v-1612741426,NaN
1,"[{'id': '4bf58dd8d48988d1ce941735', 'name': 'S...",False,4c335e83ac0ab7136bbb1c1e,1 Central Wharf Boston,US,Boston,United States,NaN,730,"[1 Central Wharf Boston, Boston, MA 02110]","[{'label': 'display', 'lat': 42.35917282104492...",42.359173,-71.049538,02110,MA,Dog n' Claw,v-1612741426,NaN
2,"[{'id': '4bf58dd8d48988d1e5941735', 'name': 'D...",False,4bc7176114d79521e3ad66e9,1277 Washington Street,US,Boston,United States,East Berkeley,2104,"[1277 Washington Street (East Berkeley), Bosto...","[{'label': 'display', 'lat': 42.34269554511267...",42.342696,-71.067776,02118,MA,Joe Wex Dog Recreation Space,v-1612741426,NaN
3,"[{'id': '5453de49498eade8af355881', 'name': 'B...",False,59362c81d0a14958ac92bd26,8 Faneuil Hall Market Pl Fl 3,US,Boston,United States,NaN,62,"[8 Faneuil Hall Market Pl Fl 3, Boston, MA 02109]","[{'label': 'display', 'lat': 42.35988214932819...",42.359882,-71.058862,02109,MA,My Busy Dog,v-1612741426,NaN
4,"[{'id': '4bf58dd8d48988d14e941735', 'name': 'A...",False,4c63048beb82d13a60d206d6,NaN,US,Boston,United States,Temple place and washington ave,194,"[Boston, MA]","[{'label': 'display', 'lat': 42.35850565365003...",42.358506,-71.058137,NaN,MA,Hot Dog Stand,v-1612741426,NaN


In [348]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns_dog = ['name', 'categories'] + [col for col in dataframe_dog.columns if col.startswith('location.')] + ['id']
dataframe_filtered_dog = dataframe_dog.loc[:, filtered_columns_dog]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered_dog['categories'] = dataframe_filtered_dog.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered_dog.columns = [column.split('.')[-1] for column in dataframe_filtered_dog.columns]

dataframe_filtered_dog.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Salty Dog Seafood Grille & Bar,Seafood Restaurant,206 Faneuil Hall Market Pl,US,Boston,United States,NaN,217,"[206 Faneuil Hall Market Pl, Boston, MA 02109]","[{'label': 'display', 'lat': 42.35993714563105...",42.359937,-71.055677,02109,MA,40b28c80f964a520b5f71ee3
1,Dog n' Claw,Seafood Restaurant,1 Central Wharf Boston,US,Boston,United States,NaN,730,"[1 Central Wharf Boston, Boston, MA 02110]","[{'label': 'display', 'lat': 42.35917282104492...",42.359173,-71.049538,02110,MA,4c335e83ac0ab7136bbb1c1e
2,Joe Wex Dog Recreation Space,Dog Run,1277 Washington Street,US,Boston,United States,East Berkeley,2104,"[1277 Washington Street (East Berkeley), Bosto...","[{'label': 'display', 'lat': 42.34269554511267...",42.342696,-71.067776,02118,MA,4bc7176114d79521e3ad66e9
3,My Busy Dog,Business Service,8 Faneuil Hall Market Pl Fl 3,US,Boston,United States,NaN,62,"[8 Faneuil Hall Market Pl Fl 3, Boston, MA 02109]","[{'label': 'display', 'lat': 42.35988214932819...",42.359882,-71.058862,02109,MA,59362c81d0a14958ac92bd26
4,Hot Dog Stand,American Restaurant,NaN,US,Boston,United States,Temple place and washington ave,194,"[Boston, MA]","[{'label': 'display', 'lat': 42.35850565365003...",42.358506,-71.058137,NaN,MA,4c63048beb82d13a60d206d6


#### Create a map

In [349]:
venues_map_dog = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around Boston


# add the parks as blue circle markers
for lat, lng, label in zip(dataframe_filtered_dog.lat, dataframe_filtered_dog.lng, dataframe_filtered_dog.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map_dog)

# display map
venues_map_dog

#### Search for entries containing 'Outdoors'

In [350]:
# SEARCHING FOR OUTDOORS NEAR BOSTON

#search_query = ['Outdoors']
limit = 500
radius = 7000
url = 'https://api.foursquare.com/v2/venues/search?categoryId=4d4b7105d754a06377d81259&client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&radius={}&limit=500'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?categoryId=4d4b7105d754a06377d81259&client_id=NAF0ED02AESTUBSNE5LS34O4SBSAGDR0NX2V04Y5VJOQLTPM&client_secret=CQ5H2IZSYIMNI4B1TCYYHOCUFEJMRMBFHPHG2ICKNUH5V5FT&ll=42.3602534,-71.0582912&oauth_token=NYUYCST5B1MIDLB1Y5DS44K4ZOJMKQMX0HV2I5CP4ZSONWPX&v=20180604&radius=7000&limit=500'

In [351]:
# GETTING THE JSON FILE

results_outdoors = requests.get(url).json()
results_outdoors

{'meta': {'code': 200, 'requestId': '60207b4314637b3b51498985'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'venues': [{'id': '4aca3daaf964a520fcc020e3',
    'name': 'One Western Avenue',
    'location': {'address': '1 Western Ave',
     'lat': 42.36460253791613,
     'lng': -71.11879588132827,
     'labeledLatLngs': [{'label': 'display',
       'lat': 42.36460253791613,
       'lng': -71.11879588132827},
      {'label': 'entrance', 'lat': 42.364241, 'lng': -71.119142}],
     'distance': 5000,
     'postalCode': '02163',
     'cc': 'US',
     'city': 'Boston',
     'state': 'MA',
     'country': 'United States',
     'formattedAddress': ['1 Western Ave', 'Boston, MA 02163']},
    'categories': [{'id': '4d954b06a243a5684965b473',
      'name': 'Residential Building (Apartment / Condo)',
      'pluralName': 'Residential Buildings (Apartments / Condos)',
      'shortName': 'Residential',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/ca

#### Creating a dataframe

In [352]:
# assign relevant part of JSON to venues
venues_outdoors = results_outdoors['response']['venues']

# tranform venues into a dataframe
dataframe_outdoors = json_normalize(venues_outdoors)
dataframe_outdoors.head()

,categories,delivery.id,delivery.provider.icon.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.name,delivery.url,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4d954b06a243a5684965b473', 'name': 'R...",NaN,NaN,NaN,NaN,NaN,NaN,False,4aca3daaf964a520fcc020e3,1 Western Ave,US,Boston,United States,NaN,5000,"[1 Western Ave, Boston, MA 02163]","[{'label': 'display', 'lat': 42.36460253791613...",42.364603,-71.118796,NaN,02163,MA,One Western Avenue,v-1612741443,NaN
1,"[{'id': '4deefb944765f83613cdba6e', 'name': 'H...",NaN,NaN,NaN,NaN,NaN,NaN,False,4b3d563ff964a520a49225e3,1 S Market St,US,Boston,United States,at Congress St & North St,157,"[1 S Market St (at Congress St & North St), Bo...","[{'label': 'display', 'lat': 42.35997750557642...",42.359978,-71.056410,Downtown Boston,02109,MA,Faneuil Hall Marketplace,v-1612741443,NaN
2,"[{'id': '4bf58dd8d48988d164941735', 'name': 'P...",NaN,NaN,NaN,NaN,NaN,NaN,False,4b6384aff964a5203d802ae3,NaN,US,Boston,United States,NaN,2632,"[Boston, MA 02118]","[{'label': 'display', 'lat': 42.33947866997181...",42.339479,-71.073568,NaN,02118,MA,Blackstone Square,v-1612741443,NaN
3,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",NaN,NaN,NaN,NaN,NaN,NaN,False,4283ee00f964a520dd221fe3,Tremont St,US,Boston,United States,btwn Beacon & Charles St,758,"[Tremont St (btwn Beacon & Charles St), Boston...","[{'label': 'display', 'lat': 42.355487, 'lng':...",42.355487,-71.064882,Downtown,02111,MA,Boston Common,v-1612741443,NaN
4,"[{'id': '4bf58dd8d48988d1df941735', 'name': 'B...",NaN,NaN,NaN,NaN,NaN,NaN,False,4a721a76f964a5205bda1fe3,Longfellow Bridge,US,Cambridge,United States,NaN,1465,"[Longfellow Bridge, Cambridge, MA 02142]","[{'label': 'display', 'lat': 42.36146689827662...",42.361467,-71.076035,NaN,02142,MA,Longfellow Bridge,v-1612741443,NaN


In [353]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns_outdoors = ['name', 'categories'] + [col for col in dataframe_outdoors.columns if col.startswith('location.')] + ['id']
dataframe_filtered_outdoors = dataframe_outdoors.loc[:, filtered_columns_outdoors]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered_outdoors['categories'] = dataframe_filtered_outdoors.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered_outdoors.columns = [column.split('.')[-1] for column in dataframe_filtered_outdoors.columns]

dataframe_filtered_outdoors.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,One Western Avenue,Residential Building (Apartment / Condo),1 Western Ave,US,Boston,United States,NaN,5000,"[1 Western Ave, Boston, MA 02163]","[{'label': 'display', 'lat': 42.36460253791613...",42.364603,-71.118796,NaN,02163,MA,4aca3daaf964a520fcc020e3
1,Faneuil Hall Marketplace,Historic Site,1 S Market St,US,Boston,United States,at Congress St & North St,157,"[1 S Market St (at Congress St & North St), Bo...","[{'label': 'display', 'lat': 42.35997750557642...",42.359978,-71.056410,Downtown Boston,02109,MA,4b3d563ff964a520a49225e3
2,Blackstone Square,Plaza,NaN,US,Boston,United States,NaN,2632,"[Boston, MA 02118]","[{'label': 'display', 'lat': 42.33947866997181...",42.339479,-71.073568,NaN,02118,MA,4b6384aff964a5203d802ae3
3,Boston Common,Park,Tremont St,US,Boston,United States,btwn Beacon & Charles St,758,"[Tremont St (btwn Beacon & Charles St), Boston...","[{'label': 'display', 'lat': 42.355487, 'lng':...",42.355487,-71.064882,Downtown,02111,MA,4283ee00f964a520dd221fe3
4,Longfellow Bridge,Bridge,Longfellow Bridge,US,Cambridge,United States,NaN,1465,"[Longfellow Bridge, Cambridge, MA 02142]","[{'label': 'display', 'lat': 42.36146689827662...",42.361467,-71.076035,NaN,02142,MA,4a721a76f964a5205bda1fe3


#### Create a map

In [354]:
venues_map_outdoors = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around Boston


# add the parks as blue circle markers
for lat, lng, label in zip(dataframe_filtered_outdoors.lat, dataframe_filtered_outdoors.lng, dataframe_filtered_outdoors.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map_outdoors)

# display map
venues_map_outdoors

#### Search for entries containing 'Playground'

In [355]:
# SEARCHING FOR PLAYGROUND NEAR BOSTON

search_query = ['Playground']
limit = 500
radius = 7000
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit=500'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
url

"https://api.foursquare.com/v2/venues/search?client_id=NAF0ED02AESTUBSNE5LS34O4SBSAGDR0NX2V04Y5VJOQLTPM&client_secret=CQ5H2IZSYIMNI4B1TCYYHOCUFEJMRMBFHPHG2ICKNUH5V5FT&ll=42.3602534,-71.0582912&oauth_token=NYUYCST5B1MIDLB1Y5DS44K4ZOJMKQMX0HV2I5CP4ZSONWPX&v=20180604&query=['Playground']&radius=7000&limit=500"

In [356]:
# GETTING THE JSON FILE

results_playground = requests.get(url).json()
results_playground

{'meta': {'code': 200, 'requestId': '60207b55fc223506bc574f5f'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'venues': [{'id': '4bc5e7ad42419521b833031d',
    'name': 'DeFilippo Playground (The Gassy)',
    'location': {'address': '135 Prince St.',
     'crossStreet': 'at Snow Hill St.',
     'lat': 42.366193782103544,
     'lng': -71.05748714736725,
     'labeledLatLngs': [{'label': 'display',
       'lat': 42.366193782103544,
       'lng': -71.05748714736725}],
     'distance': 664,
     'postalCode': '02113',
     'cc': 'US',
     'city': 'Boston',
     'state': 'MA',
     'country': 'United States',
     'formattedAddress': ['135 Prince St. (at Snow Hill St.)',
      'Boston, MA 02113']},
    'categories': [{'id': '4bf58dd8d48988d1e7941735',
      'name': 'Playground',
      'pluralName': 'Playgrounds',
      'shortName': 'Playground',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/playground_',
      

#### Create a dataframe

In [357]:
# assign relevant part of JSON to venues
venues_playground = results_playground['response']['venues']

# tranform venues into a dataframe
dataframe_playground = json_normalize(venues_playground)
dataframe_playground.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d1e7941735', 'name': 'P...",False,4bc5e7ad42419521b833031d,135 Prince St.,US,Boston,United States,at Snow Hill St.,664,"[135 Prince St. (at Snow Hill St.), Boston, MA...","[{'label': 'display', 'lat': 42.36619378210354...",42.366194,-71.057487,02113,MA,DeFilippo Playground (The Gassy),v-1612741461
1,"[{'id': '4bf58dd8d48988d1e7941735', 'name': 'P...",False,4b6f0668f964a520b8d72ce3,Main St & Eden St,US,Charlestown,United States,NaN,2279,"[Main St & Eden St, Charlestown, MA 02129]","[{'label': 'display', 'lat': 42.37932769170876...",42.379328,-71.068389,02129,MA,Edwards Playground,v-1612741461
2,"[{'id': '4bf58dd8d48988d1e7941735', 'name': 'P...",False,4c03f697f56c2d7f35931d66,Between Fairfield and Massachusetts Ave,US,Boston,United States,NaN,2532,"[Between Fairfield and Massachusetts Ave, Bost...","[{'label': 'display', 'lat': 42.35253746462541...",42.352537,-71.087252,NaN,MA,Miriam and Sydney Stoneman Playground,v-1612741461
3,"[{'id': '4bf58dd8d48988d1e7941735', 'name': 'P...",False,4c5ef8af17f3b71354df25e6,Union Park St.,US,Boston,United States,at Albany St,2258,"[Union Park St. (at Albany St), Boston, MA]","[{'label': 'display', 'lat': 42.342413, 'lng':...",42.342413,-71.071359,NaN,MA,Union Park St. Playground,v-1612741461
4,"[{'id': '4bf58dd8d48988d1e7941735', 'name': 'P...",False,4c1a2576fe5a76b0f1990415,NaN,US,Boston,United States,NaN,748,"[Boston, MA 02114]","[{'label': 'display', 'lat': 42.36043301934745...",42.360433,-71.067386,02114,MA,Phillips Street Playground,v-1612741461


In [358]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns_playground = ['name', 'categories'] + [col for col in dataframe_playground.columns if col.startswith('location.')] + ['id']
dataframe_filtered_playground = dataframe_playground.loc[:, filtered_columns_playground]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered_playground['categories'] = dataframe_filtered_playground.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered_playground.columns = [column.split('.')[-1] for column in dataframe_filtered_playground.columns]

dataframe_filtered_playground.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,DeFilippo Playground (The Gassy),Playground,135 Prince St.,US,Boston,United States,at Snow Hill St.,664,"[135 Prince St. (at Snow Hill St.), Boston, MA...","[{'label': 'display', 'lat': 42.36619378210354...",42.366194,-71.057487,02113,MA,4bc5e7ad42419521b833031d
1,Edwards Playground,Playground,Main St & Eden St,US,Charlestown,United States,NaN,2279,"[Main St & Eden St, Charlestown, MA 02129]","[{'label': 'display', 'lat': 42.37932769170876...",42.379328,-71.068389,02129,MA,4b6f0668f964a520b8d72ce3
2,Miriam and Sydney Stoneman Playground,Playground,Between Fairfield and Massachusetts Ave,US,Boston,United States,NaN,2532,"[Between Fairfield and Massachusetts Ave, Bost...","[{'label': 'display', 'lat': 42.35253746462541...",42.352537,-71.087252,NaN,MA,4c03f697f56c2d7f35931d66
3,Union Park St. Playground,Playground,Union Park St.,US,Boston,United States,at Albany St,2258,"[Union Park St. (at Albany St), Boston, MA]","[{'label': 'display', 'lat': 42.342413, 'lng':...",42.342413,-71.071359,NaN,MA,4c5ef8af17f3b71354df25e6
4,Phillips Street Playground,Playground,NaN,US,Boston,United States,NaN,748,"[Boston, MA 02114]","[{'label': 'display', 'lat': 42.36043301934745...",42.360433,-71.067386,02114,MA,4c1a2576fe5a76b0f1990415


#### Create a map

In [359]:
venues_map_playground = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around Boston


# add the parks as blue circle markers
for lat, lng, label in zip(dataframe_filtered_playground.lat, dataframe_filtered_playground.lng, dataframe_filtered_playground.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map_playground)

# display map
venues_map_playground

#### Drop 'neighborhood' column

In [360]:
dataframe_filtered_park = dataframe_filtered_park.drop(['neighborhood'], axis=1)
dataframe_filtered_outdoors = dataframe_filtered_outdoors.drop(['neighborhood'], axis=1)

#### Merging dataframes

In [361]:
df_compiled = dataframe_filtered_park.append([dataframe_filtered_pet, dataframe_filtered_dog, dataframe_filtered_outdoors,
                                             dataframe_filtered_playground])
df_compiled.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,North End Park,Park,Cross St.,US,Boston,United States,btwn North & Sudbury,290,"[Cross St. (btwn North & Sudbury), Boston, MA ...","[{'label': 'display', 'lat': 42.36248823184806...",42.362488,-71.056477,02113,MA,4ba50617f964a520f1d138e3
1,Christopher Columbus Park,Park,Atlantic Ave.,US,Boston,United States,at Atlantic Ave.,569,"[Atlantic Ave. (at Atlantic Ave.), Boston, MA ...","[{'label': 'display', 'lat': 42.36102715769776...",42.361027,-71.051451,02110,MA,4ae3580af964a520c59321e3
2,Harbor Park,Park,NaN,US,Boston,United States,NaN,448,"[Boston, MA]","[{'label': 'display', 'lat': 42.36072318013166...",42.360723,-71.052873,NaN,MA,54a1d1ce498e248911805297
3,Paul Revere Park,Park,NaN,US,Charlestown,United States,Chelsea St,1143,"[Charlestown, MA 02129]","[{'label': 'display', 'lat': 42.37019699121477...",42.370197,-71.061781,02129,MA,4bcf72669854d13a8f13f64d
4,North Point Park,Park,Museum Way,US,Cambridge,United States,NaN,1332,"[Museum Way, Cambridge, MA 02141]","[{'label': 'display', 'lat': 42.36911761365681...",42.369118,-71.069175,02141,MA,4bd9bd3fd2cbc92844a3d1ad


#### Finding shapes of the dataframes

In [362]:
df_compiled.shape

(233, 15)

In [363]:
dataframe_filtered_park.shape

(50, 15)

In [364]:
dataframe_filtered_pet.shape

(50, 15)

In [365]:
dataframe_filtered_dog.shape

(50, 15)

In [366]:
dataframe_filtered_outdoors.shape

(33, 15)

In [367]:
dataframe_filtered_playground.shape

(50, 15)

#### Dataframe for compiled data

In [368]:
df_compiled_new = df_compiled.drop(['formattedAddress', 'labeledLatLngs', 'id'], axis=1)
df_compiled_new

,name,categories,address,cc,city,country,crossStreet,distance,lat,lng,postalCode,state
0,North End Park,Park,Cross St.,US,Boston,United States,btwn North & Sudbury,290,42.362488,-71.056477,02113,MA
1,Christopher Columbus Park,Park,Atlantic Ave.,US,Boston,United States,at Atlantic Ave.,569,42.361027,-71.051451,02110,MA
2,Harbor Park,Park,NaN,US,Boston,United States,NaN,448,42.360723,-71.052873,NaN,MA
3,Paul Revere Park,Park,NaN,US,Charlestown,United States,Chelsea St,1143,42.370197,-71.061781,02129,MA
4,North Point Park,Park,Museum Way,US,Cambridge,United States,NaN,1332,42.369118,-71.069175,02141,MA
5,Peters Park,Dog Run,1205 Washington,US,Boston,United States,Berkeley,2105,42.342662,-71.067686,02118,MA
6,Piers Park,Park,95 Marginal St,US,Boston,United States,at Cottage St.,1890,42.365111,-71.036263,02128,MA
7,Langone Park,Playground,542 Commercial St,US,Boston,United States,btwn Charter & Foster,952,42.368598,-71.055734,02109,MA
8,Mary O'Malley Waterfront Park,Park,Admiral's Hill,US,Chelsea,United States,NaN,3271,42.388869,-71.049211,02150,MA
9,LoPresti Park,Park,Sumner St.,US,East Boston,United States,btwn Sumner & New,1665,42.370050,-71.042984,02128,MA


In [369]:
df_compiled_new = df_compiled_new.reset_index(drop=True)
df_compiled_new

,name,categories,address,cc,city,country,crossStreet,distance,lat,lng,postalCode,state
0,North End Park,Park,Cross St.,US,Boston,United States,btwn North & Sudbury,290,42.362488,-71.056477,02113,MA
1,Christopher Columbus Park,Park,Atlantic Ave.,US,Boston,United States,at Atlantic Ave.,569,42.361027,-71.051451,02110,MA
2,Harbor Park,Park,NaN,US,Boston,United States,NaN,448,42.360723,-71.052873,NaN,MA
3,Paul Revere Park,Park,NaN,US,Charlestown,United States,Chelsea St,1143,42.370197,-71.061781,02129,MA
4,North Point Park,Park,Museum Way,US,Cambridge,United States,NaN,1332,42.369118,-71.069175,02141,MA
5,Peters Park,Dog Run,1205 Washington,US,Boston,United States,Berkeley,2105,42.342662,-71.067686,02118,MA
6,Piers Park,Park,95 Marginal St,US,Boston,United States,at Cottage St.,1890,42.365111,-71.036263,02128,MA
7,Langone Park,Playground,542 Commercial St,US,Boston,United States,btwn Charter & Foster,952,42.368598,-71.055734,02109,MA
8,Mary O'Malley Waterfront Park,Park,Admiral's Hill,US,Chelsea,United States,NaN,3271,42.388869,-71.049211,02150,MA
9,LoPresti Park,Park,Sumner St.,US,East Boston,United States,btwn Sumner & New,1665,42.370050,-71.042984,02128,MA


#### Choosing categories related to dogs

In [370]:
stuffIwant = ['Park', 'Dog Run', 'Playground', 'Veterinarian', 'Pet Store', 
              'Spa', 'Pet Service', 'Medical Center', 'Other Great Outdoors', 'Lake', 'Beach']
stuffIwant

['Park',
 'Dog Run',
 'Playground',
 'Veterinarian',
 'Pet Store',
 'Spa',
 'Pet Service',
 'Medical Center',
 'Other Great Outdoors',
 'Lake',
 'Beach']

#### Cleaning up dataframe

In [371]:
df_cleaned = df_compiled_new.loc[df_compiled_new['categories'].isin(stuffIwant)]
df_cleaned

,name,categories,address,cc,city,country,crossStreet,distance,lat,lng,postalCode,state
0,North End Park,Park,Cross St.,US,Boston,United States,btwn North & Sudbury,290,42.362488,-71.056477,02113,MA
1,Christopher Columbus Park,Park,Atlantic Ave.,US,Boston,United States,at Atlantic Ave.,569,42.361027,-71.051451,02110,MA
2,Harbor Park,Park,NaN,US,Boston,United States,NaN,448,42.360723,-71.052873,NaN,MA
3,Paul Revere Park,Park,NaN,US,Charlestown,United States,Chelsea St,1143,42.370197,-71.061781,02129,MA
4,North Point Park,Park,Museum Way,US,Cambridge,United States,NaN,1332,42.369118,-71.069175,02141,MA
5,Peters Park,Dog Run,1205 Washington,US,Boston,United States,Berkeley,2105,42.342662,-71.067686,02118,MA
6,Piers Park,Park,95 Marginal St,US,Boston,United States,at Cottage St.,1890,42.365111,-71.036263,02128,MA
7,Langone Park,Playground,542 Commercial St,US,Boston,United States,btwn Charter & Foster,952,42.368598,-71.055734,02109,MA
8,Mary O'Malley Waterfront Park,Park,Admiral's Hill,US,Chelsea,United States,NaN,3271,42.388869,-71.049211,02150,MA
9,LoPresti Park,Park,Sumner St.,US,East Boston,United States,btwn Sumner & New,1665,42.370050,-71.042984,02128,MA


In [372]:
df_cleaned.shape

(179, 12)

#### Final dataframe

In [373]:
df_cleaned.reset_index(drop=True)

,name,categories,address,cc,city,country,crossStreet,distance,lat,lng,postalCode,state
0,North End Park,Park,Cross St.,US,Boston,United States,btwn North & Sudbury,290,42.362488,-71.056477,02113,MA
1,Christopher Columbus Park,Park,Atlantic Ave.,US,Boston,United States,at Atlantic Ave.,569,42.361027,-71.051451,02110,MA
2,Harbor Park,Park,NaN,US,Boston,United States,NaN,448,42.360723,-71.052873,NaN,MA
3,Paul Revere Park,Park,NaN,US,Charlestown,United States,Chelsea St,1143,42.370197,-71.061781,02129,MA
4,North Point Park,Park,Museum Way,US,Cambridge,United States,NaN,1332,42.369118,-71.069175,02141,MA
5,Peters Park,Dog Run,1205 Washington,US,Boston,United States,Berkeley,2105,42.342662,-71.067686,02118,MA
6,Piers Park,Park,95 Marginal St,US,Boston,United States,at Cottage St.,1890,42.365111,-71.036263,02128,MA
7,Langone Park,Playground,542 Commercial St,US,Boston,United States,btwn Charter & Foster,952,42.368598,-71.055734,02109,MA
8,Mary O'Malley Waterfront Park,Park,Admiral's Hill,US,Chelsea,United States,NaN,3271,42.388869,-71.049211,02150,MA
9,LoPresti Park,Park,Sumner St.,US,East Boston,United States,btwn Sumner & New,1665,42.370050,-71.042984,02128,MA


#### Creating a map for dog related venues

In [374]:
dog_venues_map = folium.Map(location=[latitude, longitude], zoom_start=12) # generate map centred around Boston


# add the venues as blue circle markers
for lat, lng, label in zip(df_cleaned.lat, df_cleaned.lng, df_cleaned.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(dog_venues_map)

# display map
dog_venues_map

#### Map for venues by category

In [376]:
colors = ['red', 'blue', 'green', 'purple', 'orange', 'beige', 'darkpurple', 'white', 'gray', 'pink', 'darkgreen']

dog_category_map = folium.Map(location=[latitude, longitude], zoom_start=12) # generate map centred around Boston

for i,j in enumerate(colors):
    df_new = df_cleaned.loc[df_cleaned['categories']==stuffIwant[i]]
    for lat, lng, label in zip(df_new.lat, df_new.lng, df_new.categories): 
        folium.CircleMarker(
        [lat, lng],
        radius=5,
        color=j,
        popup=label,
        fill = True,
        fill_color=j,
        fill_opacity=0.6
        ).add_to(dog_category_map)
    
dog_category_map